# Azure AI Search Connection Setup - Interactive Tutorial

**Module 1: Introduction and Setup**

Welcome to your first hands-on experience with Azure AI Search! This interactive notebook will guide you through the process of connecting to Azure AI Search service step by step.

## 🎯 Learning Objectives

By the end of this notebook, you will:
- Understand how to authenticate with Azure AI Search
- Learn to create and configure search clients
- Practice connection testing and troubleshooting
- Explore different authentication methods
- Discover available indexes and service information

## 📋 Prerequisites

Before starting, make sure you have:
- ✅ Azure AI Search service created in Azure portal
- ✅ Service endpoint URL and API key
- ✅ Environment variables configured in `.env` file
- ✅ Required Python packages installed (`pip install -r requirements.txt`)

## 🚀 Let's Get Started!

Run each cell in order and follow the explanations. Don't hesitate to experiment with the code!

## Step 1: Import Required Libraries

First, let's import all the libraries we'll need for this tutorial. Each import serves a specific purpose in connecting to Azure AI Search.

In [ ]:
# Core Azure AI Search libraries
from azure.search.documents import SearchClient          # For document operations (search, add, update, delete)
from azure.search.documents.indexes import SearchIndexClient  # For index management operations
from azure.core.credentials import AzureKeyCredential   # For API key authentication
from azure.identity import DefaultAzureCredential       # For managed identity authentication
from azure.core.exceptions import AzureError, ClientAuthenticationError  # For error handling

# Standard Python libraries
import os
import json
import logging
from typing import Optional, Dict, Any, List
from datetime import datetime

# Environment and utility libraries
from dotenv import load_dotenv  # For loading environment variables from .env file

print("✅ All libraries imported successfully!")
print("📚 You're now ready to connect to Azure AI Search")

## Step 2: Load Configuration

Let's load our configuration from environment variables. This is a secure way to store sensitive information like API keys.

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Load configuration from environment variables
config = {
    'endpoint': os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT'),
    'api_key': os.getenv('AZURE_SEARCH_API_KEY'),
    'index_name': os.getenv('AZURE_SEARCH_INDEX_NAME', 'sample-index'),
    'use_managed_identity': os.getenv('USE_MANAGED_IDENTITY', 'false').lower() == 'true'
}

# Display configuration (hiding sensitive data)
print("📋 Configuration loaded:")
for key, value in config.items():
    if key == 'api_key' and value:
        print(f"   {key}: {'*' * 8}...{value[-4:] if len(value) > 4 else '***'}")
    else:
        print(f"   {key}: {value}")

# Validate required configuration
if not config['endpoint']:
    print("❌ Missing AZURE_SEARCH_SERVICE_ENDPOINT in environment variables")
elif not config['api_key'] and not config['use_managed_identity']:
    print("❌ Missing AZURE_SEARCH_API_KEY in environment variables")
else:
    print("✅ Configuration looks good!")

## Step 3: Create Your First Search Client

Now let's create a SearchClient, which is used for document operations like searching, adding, updating, and deleting documents.

In [ ]:
def create_search_client(index_name: Optional[str] = None) -> Optional[SearchClient]:
    """
    Create a SearchClient for document operations.
    
    Args:
        index_name: Name of the index to connect to (optional)
        
    Returns:
        SearchClient or None if creation failed
    """
    try:
        # Use provided index name or fall back to configuration
        target_index = index_name or config['index_name']
        
        print(f"🔧 Creating SearchClient for index: {target_index}")
        
        # Validate required parameters
        if not config['endpoint']:
            raise ValueError("Service endpoint is required")
        if not config['api_key']:
            raise ValueError("API key is required")
        if not target_index:
            raise ValueError("Index name is required")
        
        # Create the credential object
        credential = AzureKeyCredential(config['api_key'])
        
        # Create the SearchClient
        search_client = SearchClient(
            endpoint=config['endpoint'],
            index_name=target_index,
            credential=credential
        )
        
        print(f"✅ SearchClient created successfully!")
        return search_client
        
    except Exception as e:
        print(f"❌ Failed to create SearchClient: {str(e)}")
        return None

# Create your first search client
search_client = create_search_client()

if search_client:
    print("🎉 Congratulations! You've created your first Azure AI Search client.")
else:
    print("💡 Don't worry if this failed - we'll troubleshoot together in the next steps.")

## Step 4: Create an Index Management Client

The SearchIndexClient is used for administrative operations like creating, updating, and deleting indexes.

In [ ]:
def create_index_client() -> Optional[SearchIndexClient]:
    """
    Create a SearchIndexClient for index management operations.
    
    Returns:
        SearchIndexClient or None if creation failed
    """
    try:
        print("🔧 Creating SearchIndexClient for index management...")
        
        # Validate required parameters
        if not config['endpoint']:
            raise ValueError("Service endpoint is required")
        if not config['api_key']:
            raise ValueError("API key is required")
        
        # Create the credential object
        credential = AzureKeyCredential(config['api_key'])
        
        # Create the SearchIndexClient
        index_client = SearchIndexClient(
            endpoint=config['endpoint'],
            credential=credential
        )
        
        print("✅ SearchIndexClient created successfully!")
        return index_client
        
    except Exception as e:
        print(f"❌ Failed to create SearchIndexClient: {str(e)}")
        return None

# Create the index management client
index_client = create_index_client()

if index_client:
    print("🎉 Great! You now have both types of clients for Azure AI Search.")
    print("   - SearchClient: For document operations (search, add, update, delete)")
    print("   - SearchIndexClient: For index management (create, update, delete indexes)")

## Step 5: Test Your Connection

Let's test if we can successfully connect to your Azure AI Search service by retrieving service statistics.

In [ ]:
def test_connection() -> bool:
    """
    Test the connection to Azure AI Search service.
    
    Returns:
        True if connection successful, False otherwise
    """
    print("🧪 Testing connection to Azure AI Search...")
    
    try:
        if not index_client:
            print("❌ No index client available for testing")
            return False
        
        # Attempt to get service statistics (lightweight operation)
        stats = index_client.get_service_statistics()
        
        # Display success message with service information
        print("✅ Connection test successful!")
        print("📊 Your Azure AI Search Service Statistics:")
        print(f"   📄 Total Documents: {stats.counters.document_count:,}")
        print(f"   📚 Total Indexes: {stats.counters.index_count}")
        print(f"   🗂️  Total Indexers: {stats.counters.indexer_count}")
        print(f"   🔗 Total Data Sources: {stats.counters.data_source_count}")
        print(f"   💾 Storage Used: {stats.counters.storage_size:,} bytes")
        
        print("\n📏 Service Limits:")
        print(f"   📚 Max Indexes: {stats.limits.max_indexes_allowed}")
        print(f"   🏷️  Max Fields per Index: {stats.limits.max_fields_per_index}")
        
        return True
        
    except ClientAuthenticationError as e:
        print("❌ Authentication failed - please check your API key")
        print(f"   Error details: {str(e)}")
        print("\n💡 Troubleshooting tips:")
        print("   1. Verify your API key is correct in the .env file")
        print("   2. Check if the API key has the necessary permissions")
        print("   3. Ensure the key hasn't expired")
        return False
        
    except AzureError as e:
        print(f"❌ Azure service error: {str(e)}")
        print("\n💡 Troubleshooting tips:")
        print("   1. Check if your Azure AI Search service is running")
        print("   2. Verify the service endpoint URL is correct")
        print("   3. Ensure your Azure subscription is active")
        return False
        
    except Exception as e:
        print(f"❌ Unexpected error during connection test: {str(e)}")
        print("\n💡 General troubleshooting:")
        print("   1. Check your internet connection")
        print("   2. Verify all environment variables are set correctly")
        print("   3. Try running the validation script: python setup/validate_setup.py")
        return False

# Test the connection
connection_successful = test_connection()

if connection_successful:
    print("\n🎉 Excellent! Your connection to Azure AI Search is working perfectly.")
    print("You're ready to start exploring search operations!")
else:
    print("\n🔧 Don't worry - connection issues are common when getting started.")
    print("Let's continue with the tutorial to learn more troubleshooting techniques.")

## Step 6: Discover Available Indexes

Let's see what indexes are available in your Azure AI Search service. This will help us understand what data we can work with.

In [ ]:
def list_available_indexes() -> List[str]:
    """
    List all available indexes in the search service.
    
    Returns:
        List of index names
    """
    print("📋 Discovering available indexes in your search service...")
    
    try:
        if not index_client:
            print("❌ No index client available")
            return []
        
        # Get list of indexes
        indexes = list(index_client.list_indexes())
        index_names = [index.name for index in indexes]
        
        if index_names:
            print(f"✅ Found {len(index_names)} index(es) in your search service:")
            for i, name in enumerate(index_names, 1):
                print(f"   {i}. {name}")
                
            # Show additional details for the first index
            if indexes:
                first_index = indexes[0]
                print(f"\n📊 Details for '{first_index.name}':")
                print(f"   🏷️  Fields: {len(first_index.fields)}")
                print(f"   🔍 Suggesters: {len(first_index.suggesters or [])}")
                print(f"   📝 Scoring Profiles: {len(first_index.scoring_profiles or [])}")
        else:
            print("ℹ️  No indexes found in your search service.")
            print("💡 This is normal for a new service. You can create indexes in later modules.")
        
        return index_names
        
    except Exception as e:
        print(f"❌ Failed to list indexes: {str(e)}")
        return []

# Discover available indexes
available_indexes = list_available_indexes()

# Store for later use
print(f"\n📝 Found {len(available_indexes)} indexes that we can work with in this tutorial.")

## Step 7: Test Index Access (If Available)

If we found any indexes, let's test accessing one of them to see how many documents it contains.

In [ ]:
def test_index_access(index_name: str) -> bool:
    """
    Test access to a specific index.
    
    Args:
        index_name: Name of the index to test
        
    Returns:
        True if index access successful, False otherwise
    """
    print(f"🎯 Testing access to index: '{index_name}'")
    
    try:
        # Create a search client for this specific index
        test_client = create_search_client(index_name)
        
        if not test_client:
            print("❌ Could not create search client for this index")
            return False
        
        # Get document count (simple operation)
        document_count = test_client.get_document_count()
        
        print(f"✅ Successfully accessed index '{index_name}'!")
        print(f"📄 This index contains {document_count:,} documents")
        
        if document_count > 0:
            print("🎉 Great! This index has data we can search through in later modules.")
        else:
            print("ℹ️  This index is empty, but that's okay - we can add data in later modules.")
        
        return True
        
    except Exception as e:
        print(f"❌ Failed to access index '{index_name}': {str(e)}")
        print("\n💡 This might happen if:")
        print("   - The index name is incorrect")
        print("   - Your API key doesn't have permission to access this index")
        print("   - The index was recently created and isn't ready yet")
        return False

# Test index access if we have available indexes
if available_indexes:
    print("🧪 Let's test access to your first available index...\n")
    index_access_successful = test_index_access(available_indexes[0])
    
    if index_access_successful:
        print("\n🎊 Perfect! You can successfully access your indexes.")
        print("This means you're ready for document operations in the next modules.")
else:
    print("ℹ️  No indexes available to test, but that's perfectly fine!")
    print("We'll learn how to create indexes in Module 3: Index Management.")
    index_access_successful = None

## Step 8: Advanced Authentication (Optional)

Let's explore managed identity authentication, which is the recommended approach for production scenarios.

In [ ]:
def demonstrate_managed_identity() -> bool:
    """
    Demonstrate connection using managed identity authentication.
    
    Returns:
        True if managed identity connection successful, False otherwise
    """
    print("🔐 Exploring Managed Identity Authentication...")
    print("\nℹ️  Managed Identity is the recommended authentication method for production because:")
    print("   - No need to store API keys in your code or configuration")
    print("   - Automatic credential rotation")
    print("   - Better security and compliance")
    print("   - Integration with Azure RBAC (Role-Based Access Control)")
    
    try:
        # Check if managed identity is configured
        if not config.get('use_managed_identity'):
            print("\n⏭️  Managed identity not configured in environment variables.")
            print("💡 To enable managed identity:")
            print("   1. Set USE_MANAGED_IDENTITY=true in your .env file")
            print("   2. Ensure your Azure resource has managed identity enabled")
            print("   3. Grant the identity 'Search Service Contributor' role")
            return False
        
        print("\n🔄 Attempting managed identity authentication...")
        
        # Create credential using managed identity
        credential = DefaultAzureCredential()
        
        # Create SearchIndexClient with managed identity
        mi_index_client = SearchIndexClient(
            endpoint=config['endpoint'],
            credential=credential
        )
        
        # Test the connection
        stats = mi_index_client.get_service_statistics()
        
        print("✅ Managed identity authentication successful!")
        print(f"📊 Connected using managed identity - {stats.counters.index_count} indexes available")
        print("🎉 You're using the most secure authentication method!")
        
        return True
        
    except Exception as e:
        print(f"❌ Managed identity authentication failed: {str(e)}")
        print("\n💡 Managed identity troubleshooting:")
        print("   1. Ensure you're running in an Azure environment (VM, App Service, etc.)")
        print("   2. Verify managed identity is enabled for your resource")
        print("   3. Check that the identity has 'Search Service Contributor' role")
        print("   4. For local development, use Azure CLI: 'az login'")
        
        return False

# Demonstrate managed identity (if configured)
managed_identity_successful = demonstrate_managed_identity()

if managed_identity_successful:
    print("\n🏆 Excellent! You're using enterprise-grade authentication.")
else:
    print("\n📚 Don't worry - API key authentication is perfect for learning and development.")
    print("You can explore managed identity later when you're ready for production deployments.")

## Step 9: Connection Summary and Next Steps

Let's summarize what we've accomplished and plan our next steps in the learning journey.

In [ ]:
# Create a summary of our connection tests
def generate_connection_summary():
    """
    Generate a comprehensive summary of all connection tests performed.
    """
    print("📊 Azure AI Search Connection Summary")
    print("=" * 50)
    
    # Configuration status
    config_status = "✅ Valid" if config['endpoint'] and config['api_key'] else "❌ Invalid"
    print(f"🔧 Configuration: {config_status}")
    
    # Connection status
    connection_status = "✅ Successful" if connection_successful else "❌ Failed"
    print(f"🌐 Basic Connection: {connection_status}")
    
    # Index discovery
    index_discovery = f"✅ Found {len(available_indexes)} indexes" if available_indexes else "ℹ️  No indexes found"
    print(f"📚 Index Discovery: {index_discovery}")
    
    # Index access
    if index_access_successful is True:
        index_access = "✅ Successful"
    elif index_access_successful is False:
        index_access = "❌ Failed"
    else:
        index_access = "⏭️  Skipped (no indexes)"
    print(f"🎯 Index Access: {index_access}")
    
    # Managed identity
    mi_status = "✅ Successful" if managed_identity_successful else "⏭️  Not configured"
    print(f"🔐 Managed Identity: {mi_status}")
    
    print("\n" + "=" * 50)
    
    # Overall assessment
    if connection_successful:
        print("🎉 Overall Status: READY FOR NEXT MODULE!")
        print("\n🚀 Recommended Next Steps:")
        print("   1. ✅ Complete the exercises in the exercises/ directory")
        print("   2. ✅ Move on to Module 2: Basic Search Operations")
        print("   3. ✅ Explore the interactive search examples")
        
        if available_indexes:
            print(f"   4. ✅ Practice with your existing indexes: {', '.join(available_indexes[:3])}")
        else:
            print("   4. ✅ Learn to create indexes in Module 3: Index Management")
    else:
        print("🔧 Overall Status: NEEDS TROUBLESHOOTING")
        print("\n🛠️  Recommended Troubleshooting Steps:")
        print("   1. ❗ Run the validation script: python setup/validate_setup.py")
        print("   2. ❗ Check your .env file configuration")
        print("   3. ❗ Verify your Azure AI Search service is running")
        print("   4. ❗ Confirm your API key permissions")
        print("   5. ❗ Review the troubleshooting documentation")
    
    print("\n📚 Additional Resources:")
    print("   📖 Documentation: docs/beginner/module-01-introduction-setup/documentation.md")
    print("   🐍 Python Script: code-samples/connection_setup.py")
    print("   🏋️  Exercises: exercises/")
    print("   🔍 Troubleshooting: setup/validate_setup.py")
    print("   🌐 Azure Portal: https://portal.azure.com")

# Generate the summary
generate_connection_summary()

## 🎓 Congratulations!

You've completed the Azure AI Search Connection Setup tutorial! Here's what you've learned:

### ✅ Skills Acquired
- **Authentication**: How to authenticate with Azure AI Search using API keys
- **Client Creation**: Creating SearchClient and SearchIndexClient instances
- **Connection Testing**: Validating your connection and troubleshooting issues
- **Service Discovery**: Exploring available indexes and service statistics
- **Security**: Understanding managed identity authentication for production

### 🔧 Technical Concepts
- **SearchClient**: Used for document operations (search, add, update, delete)
- **SearchIndexClient**: Used for index management (create, update, delete indexes)
- **AzureKeyCredential**: API key-based authentication
- **DefaultAzureCredential**: Managed identity authentication
- **Service Statistics**: Understanding your search service capacity and usage

### 🚀 What's Next?

Now that you can connect to Azure AI Search, you're ready to:

1. **Practice with Exercises**: Complete the hands-on exercises in the `exercises/` directory
2. **Basic Search Operations**: Move on to Module 2 to learn how to search documents
3. **Index Management**: Learn to create and manage indexes in Module 3
4. **Advanced Topics**: Explore filtering, faceting, and advanced search features

### 💡 Pro Tips for Success
- Always test your connection before starting development
- Use managed identity in production environments
- Keep your API keys secure and rotate them regularly
- Monitor your service usage and performance
- Experiment with the code - that's how you learn best!

### 🆘 Need Help?
If you encountered any issues:
- Run the validation script: `python setup/validate_setup.py`
- Check the troubleshooting documentation
- Review your environment configuration
- Practice with the exercises to reinforce your learning

**Happy searching! 🔍✨**